# Based on Issue #31: Add cost and time estimates as well
We want to add simple cost and time estimates to assess the broader impacts of programs.
We will do so by creating simple distance-based maps for each metric - e.g.

cost_per_mile = { "drove_alone": ..., "shared_ride": ...., "pilot_ebike": ....
}

We can then compute the overall impact of the metric by pseudo code similar to:

for trip in trips:
    cost_impact_trip = (cost_per_mile[“ebike”] – cost_per_mile[trip.replaced_mode]) * trip.length_in_miles
    cost_impact_trips.append(cost_impact_trip)

cost_impact_overall = sum(cost_impact_trips)

Of course, we could also use pandas if that works better - e.g. something like:

cost_impact_trips = trips.apply(lambda trip_row: (cost_per_mile["ebike"] - cost_per_mile[trip_row.replaced_mode]) * trip_row.length__in_miles
cost_impact_overall = cost_impact_trips.sum()


Shankari K. suggested following the process outlined in the energy_calculations notebook reproduced below.

In [ ]:
# user defined modules
import scaffolding
from plots import *

# external packages
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import defaultdict

In [ ]:
# global configurations
sns.set_style('whitegrid')
sns.set()

%matplotlib inline

In [ ]:
# external variables (run mapping_dictionaries notebook before running this cell)
%store -r df_EI  
%store -r df_CT

%store -r dic_re
%store -r dic_pur
%store -r dic_fuel

# convert a dictionary to a defaultdict
dic_pur = defaultdict(lambda: 'Other',dic_pur)
dic_re = defaultdict(lambda: 'Other',dic_re)

In [ ]:
# Scaffolding Inputs (None -> get all data)
year = None
month = None
program = None

In [ ]:
# Define time series for year and month
tq = scaffolding.get_time_query(year, month)

In [ ]:
# Acquire rows from database corresponding to time series and program label
participant_ct_df = scaffolding.load_all_participant_trips(program, tq)

In [ ]:
# Remove any labelled trips from the dataframe
labeled_ct = scaffolding.filter_labeled_trips(participant_ct_df)

In [ ]:
# Just expand the user_input feature to multiple features for each entry
expanded_ct = scaffolding.expand_userinputs(labeled_ct)

In [ ]:
# Removes some rows that don't show a change from another mode to pilot e-bike + name same_mode as confirmed_mode
expanded_ct = scaffolding.data_quality_check(expanded_ct)
expanded_ct.shape

In [ ]:
## Mapping new labels with dictionaries
expanded_ct['Trip_purpose']= expanded_ct['purpose_confirm'].map(dic_pur)
expanded_ct['Mode_confirm']= expanded_ct['mode_confirm'].map(dic_re)
expanded_ct['Replaced_mode']= expanded_ct['replaced_mode'].map(dic_re)

#Mapping fuel
expanded_ct['Mode_confirm_fuel']= expanded_ct['Mode_confirm'].map(dic_fuel)
expanded_ct['Replaced_mode_fuel']= expanded_ct['Replaced_mode'].map(dic_fuel)
expanded_ct.shape

In [ ]:
# Just a meters to miles conversion at this point
scaffolding.unit_conversions(expanded_ct)

In [ ]:
file_suffix = scaffolding.get_file_suffix(year, month, program)
quality_text = scaffolding.get_quality_text(participant_ct_df, expanded_ct)

In [ ]:
expanded_ct['duration_h']

### Analysis of Time Impact

In [ ]:
expanded_ct = scaffolding.cost(expanded_ct, df_CT, 'distance','Replaced_mode', 'Mode_confirm')
expanded_ct = scaffolding.cost_impact(expanded_ct, 'distance_miles','Replaced_mode', 'Mode_confirm')

In [ ]:
expanded_ct.columns

In [ ]:
data=expanded_ct.loc[(expanded_ct['distance_miles'] <= 40)].sort_values(by=['Cost_Impact($)'], ascending=False)   
x='Cost_Impact($)'
y='distance_miles'
legend ='Mode_confirm'
plot_title="Sketch of Cost_Impact($) by Travel Mode Selected\n%s" % quality_text
file_name ='sketch_distance_cost_impact%s.png' % file_suffix
distancevsenergy(data,x,y,legend,plot_title,file_name)

In [ ]:
#eirp : energy impact replaced_mode
eirc=expanded_ct.groupby('Replaced_mode').agg({'Cost_Impact($)': ['sum', 'mean']},)
eirc.columns = ['Sketch of Total Cost_Impact($)', 'Sketch of Average Cost_Impact($)']
eirc = eirc.reset_index()
eirc = eirc.sort_values(by=['Sketch of Total Cost_Impact($)'], ascending=False)
eirc['boolean'] = eirc['Sketch of Total Cost_Impact($)'] > 0

#eimc : energy impact mode_confirm
eimc=expanded_ct.groupby('Mode_confirm').agg({'Energy_Impact(kWH)': ['sum', 'mean']},)
eimc.columns = ['Sketch of Total Energy_Impact(kWH)', 'Sketch of Average Energy_Impact(kWH)']
eimc = eimc.reset_index()
eimc = eimc.sort_values(by=['Sketch of Total Energy_Impact(kWH)'], ascending=False)


subset1 = eirc [['Replaced_mode', 'Sketch of Total Energy_Impact(kWH)']].copy()
subset1.rename(columns = {'Replaced_mode':'Transport Mode','Sketch of Total Energy_Impact(kWH)':'Replaced_Mode' }, inplace=True)

subset2 = eimc [['Mode_confirm', 'Sketch of Total Energy_Impact(kWH)']].copy()
subset2.rename(columns = {'Mode_confirm':'Transport Mode','Sketch of Total Energy_Impact(kWH)':'Mode_Confirm' }, inplace=True)

df_plot = pd.merge(subset1, subset2, on="Transport Mode")
df = pd.melt(df_plot , id_vars=['Transport Mode'], value_vars=['Replaced_Mode','Mode_Confirm'], var_name='selection')
df.rename(columns = {'value':'Energy Impact (kWH)'}, inplace = True)

### Analysis of Time Impact